In [905]:
# Import dependencies
import pandas as pd
import numpy as np
import json
import re

from os import listdir
from os.path import isfile, join
from collections import Counter
from datetime import datetime
from matplotlib import pyplot as plt

import plotly.express as px

## Data Retrieval

In [906]:
# Define directory path
mypath = "./data_dir"

# Create list of json file names in directory
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [907]:
# Initialize empty list for all debate data
all_debates_list = []

# Loop through and read in all json files
for file in onlyfiles:
    
    # Open next JSON file
    with open(f"{mypath}/{file}") as json_data:

        # Load data from current JSON file
        data = json.load(json_data)

        # Fix dates for 1992 debate parts 1 and 2
        if (data['date'][0] == 'Part'):
            if data['date'][1] == '1':
                data['date'] = ['October', '11', '1992']
            elif data['date'][1] == '2':
                data['date'] = ['October', '15', '1992']

        # Append to full debate data list
        all_debates_list.append(data)

In [908]:
# Get date in datetime format from a particular debate JSON
def get_date(debate):
    
    # Create date string from provided date array
    date_string = '-'.join(debate['date'])

    # Create date object from date string
    date_object = datetime.strptime(date_string, '%B-%d-%Y')

    # Return date of given debate in datetime
    return date_object

In [909]:
# Get list of unique actors from a particular debate JSON
def get_unique_actors(debate):
    # Empty list to store all actors names (including duplicates)
    actor_list = []

    # Create list of prefixes to remove from names
    prefixes = ['Mr.', 'Ms.', 'Senator', 'Governor', 'Admiral']

    # Create dictionary of actor name corrections to be made
    typo_corrections = {
        # 'The President':'Reagan',
        '^Obam$':'Obama',
        'Barbara Walters':'Walters',
        'Bill Shadel': 'Shadel',
        'Edwin Newman': 'Newman',
        'Frank Mcgee': 'McGee',
        'Hal Bruno': 'Bruno',
        'Harry Ellis': 'Ellis',
        'Jim Lehrer': 'Lehrer',
        'Quincy Howe': 'Howe',
        'Sander Vanocur': 'Vanocur',
        'President Bush': 'Bush',
        '^Frederic$': 'Frederick'
    }

    # Loop through each speaking turn in the debate transcription content
    for turn in debate['content']:

        # Store actor name for each speaking turn
        actor = turn['actor']

        # Loop through prefixes in list
        for prefix in prefixes:

            # Check if the actor's name contains current prefix
            if prefix in actor:

                # Remove prefix and strip whitespace
                actor = actor.replace(prefix, '').strip()

        # Append actor to list of non-unique names
        actor_list.append(actor)

    # Loop through enumerated list of actor names
    for index, data in enumerate(actor_list):

        # Loop through items in typo corrections dict
        for k, v in typo_corrections.items():

            # Check if typo is present in current actor name
            if k in data:

                # Correct typo
                actor_list[index]=data.replace(k, typo_corrections[k])

    # Create unique list of cleaned actor names
    cleaned_actors = list(set(actor_list))

    # Return list of unique actor names for given debate
    return cleaned_actors

In [910]:
# Get non-unique words given actor name and debate JSON data
def get_actor_dialogue(debate, actor):

    # Initialize actor's dialogue to empty string
    filtered_dialogue = ''

    # Initialize speaking turn counter to 0
    speaking_turn_count = 0

    # Loop through speaking turns in debate JSON data
    for turn in debate['content']:
        
        # Check if speaking turn is made by specified actor
        if actor in turn['actor'].split():

            # Increment actor's speaking turn count
            speaking_turn_count += 1

            # Concatenate to running dialogue string for selected actor
            filtered_dialogue += f"{turn['dialogue']} "
    # Create nonunique word list from dialogue with punctuation and whitespace removed
    turn_dialogue_list = re.sub(r'[^\w\s]', '', filtered_dialogue).split()
    
    # Make lowercase list of all words
    dialogue_list_lower = [word.lower() for word in turn_dialogue_list]

    
    # Return actor's speaking turn count and nonunique word list
    return speaking_turn_count, dialogue_list_lower

In [911]:
### Collect data for dataframe

# Initialize row data for dataframe to empty list
row_data_list = []

# Loop through all debate data
for debate in all_debates_list:

    # Store date of currently selected debate
    debate_date = get_date(debate)

    # Loop through unique actor list for each debate
    for actor in get_unique_actors(debate):
        
        # Store speaking turn count and nonunique word list for currently selected debate/actor
        speaking_turn_count, actor_dialogue_list = get_actor_dialogue(debate, actor)

        # Store nonunique word count for selected debate/actor
        total_word_count = len(actor_dialogue_list)
    
        # Store unique word count for selected debate/actor
        unique_word_count = len(set(actor_dialogue_list))

        # Create row of relevant data for selected debate/actor
        row_data = [actor, debate_date, speaking_turn_count, total_word_count, unique_word_count]

        # Append row data to list
        row_data_list.append(row_data)

In [912]:
# Create dataframe from collected row data
debate_dialogue_df = pd.DataFrame(row_data_list, columns=['actor', 'date', 'speaking_turn_count','total_word_count', 'unique_word_count'])

# Preview dataframe
debate_dialogue_df

,actor,date,speaking_turn_count,total_word_count,unique_word_count
0,Participants,2020-09-29,1,7,7
1,Trump,2020-09-29,341,7394,1155
2,Moderator,2020-09-29,1,4,4
3,Wallace,2020-09-29,246,4711,963
4,Biden,2020-09-29,269,6529,1224
...,...,...,...,...,...
303,Obama,2008-10-07,39,7046,1344
304,Brokaw,2008-10-07,58,1703,533
305,Mccain,2008-10-07,40,6281,1270
306,Transcription By,2008-10-07,0,0,0


In [913]:
# Create new column with calculated average number of words per turn
debate_dialogue_df['avg_words_per_turn'] = debate_dialogue_df['total_word_count'] / debate_dialogue_df['speaking_turn_count']

# Create new column with calculated average number of words per turn
debate_dialogue_df['avg_unique_words_per_turn'] = debate_dialogue_df['unique_word_count'] / debate_dialogue_df['speaking_turn_count']

# Fix Reagan's name in 1984
debate_dialogue_df['actor'].replace({"The President": "Reagan"})

0          Participants
1                 Trump
2             Moderator
3               Wallace
4                 Biden
             ...       
303               Obama
304              Brokaw
305              Mccain
306    Transcription By
307          descriptor
Name: actor, Length: 308, dtype: object

In [914]:
# Export dataframe as CSV
debate_dialogue_df.to_csv('./debate_data.csv', index=False)

debate_dialogue_df.sort_values(by="unique_word_count", ascending=False)

,actor,date,speaking_turn_count,total_word_count,unique_word_count,avg_words_per_turn,avg_unique_words_per_turn
251,Kaine,2016-10-04,188,7560,1464,40.212766,7.787234
298,Lieberman,2000-10-05,34,6685,1462,196.617647,43.000000
158,Kerry,2004-10-08,37,7252,1449,196.000000,39.162162
301,Cheney,2000-10-05,30,6663,1446,222.100000,48.200000
106,Dole,1996-10-06,46,8077,1426,175.586957,31.000000
...,...,...,...,...,...,...,...
142,Audience Member,1992-10-15,0,0,0,NaN,NaN
286,McGee,1960-10-07,0,0,0,NaN,NaN
174,A Reminder,2004-10-05,0,0,0,NaN,NaN
175,The Rules,2004-10-05,0,0,0,NaN,NaN


In [915]:
# Create function to count number of times each word was used
def actor_word_count(debate, actor):
    remove_words = ['the', 'to', 'of', 'in', 'and', 'that', 'a', 'is', 'for', 'it']
    turn_count, dialogue_list = get_actor_dialogue(debate, actor)

    dialogue_list_cleaned = [word for word in dialogue_list if word not in remove_words]

    c = Counter(dialogue_list_cleaned)

    return c

In [916]:
debates_wiki_df = pd.read_html("http://en.wikipedia.org/wiki/United_States_presidential_debates")

candidates_table = debates_wiki_df[2]
viewership_table = debates_wiki_df[3]

## Data Cleaning

In [917]:
vp_candidates = pd.DataFrame()
pres_candidates = pd.DataFrame()

candidates_table = candidates_table.rename(columns={"Election":"year",
                                "Presidential debates":"pres_debate_count",
                                "Presidential debates.1":"pres_candidate",
                                "Vice presidential debates":"vp_debate_count",
                                "Vice presidential debates.1":"vp_candidate"})

candidates_cleaned_df = candidates_table.drop(index=2)

pres_candidates[['year', 'debate_count', 'candidate']] = candidates_table[['year', 'pres_debate_count','pres_candidate']]
pres_candidates['type'] = 'P'
vp_candidates[['year', 'debate_count', 'candidate']] = candidates_table[['year', 'vp_debate_count','vp_candidate']]
vp_candidates['type'] = 'VP'

In [918]:
candidates_df = pd.concat([pres_candidates, vp_candidates], ignore_index=True).sort_values(by='year').reset_index(drop=True)

candidates_df.drop(index=[56,57], inplace=True)

In [919]:
candidates_df["debate_count"][candidates_df["year"] == '2020'] = '2'

candidates_df["debate_count"][candidates_df["debate_count"].str.contains('ebate', na=False)] = 0

candidates_df.head() 

,year,debate_count,candidate,type
0,1960,4,Vice President Richard Nixon (R),P
1,1960,4,Senator John F. Kennedy (D),P
2,1960,0,No debates until 1976,VP
3,1960,0,No debates until 1976,VP
4,1976,3,President Gerald Ford (R),P


In [920]:
candidates_df[['debate_count', 'year']] = candidates_df[['debate_count', 'year']].astype(int)

In [921]:
candidates_df = candidates_df[candidates_df["candidate"].str.contains('ebate')==False]

In [922]:
candidates_df[["candidate", "party"]] = candidates_df["candidate"].str.split("(", expand=True)
candidates_df["party"] = candidates_df["party"].str[0]
candidates_df['last_name'] = candidates_df['candidate'].str.split().str[-1]
candidates_df = candidates_df.reset_index(drop=True)


In [923]:
won_election = [False, True, False, True, False, True, False, True, False, False,
                 True, True, False, False, True, True, False, False, True, False,
                 True, False, False, True, False, False, True, True, False, True,
                 False, False, False, True, True, True, False, True, False, False,
                 False, True, True, False, True, True, False, True, False, False,
                 True]

candidates_df.insert(6, 'won_election', won_election)

candidates_df.head()

,year,debate_count,candidate,type,party,last_name,won_election
0,1960,4,Vice President Richard Nixon,P,R,Nixon,False
1,1960,4,Senator John F. Kennedy,P,D,Kennedy,True
2,1976,3,President Gerald Ford,P,R,Ford,False
3,1976,3,Former Governor Jimmy Carter,P,D,Carter,True
4,1976,1,Senator Bob Dole,VP,R,Dole,False


In [924]:
debate_dialogue_df['year'] = pd.DatetimeIndex(debate_dialogue_df['date']).year

debate_dialogue_df = debate_dialogue_df.merge(candidates_df[['last_name', 'won_election']], left_on='actor', right_on='last_name')

In [925]:
# Create list of candidate titles
titles = ['Former Vice President', 'Vice President', 'President',  'Former Senator', 'Senator', 'Former Governor',
          'Governor', 'Congressman', 'Congresswoman', 'Businessman', 'Ret. Vice Admiral', 'Former HUD Secretary',
          'Former Secretary of State']

# Define function to return the title string contained in candidate name
def title_split(a):
    for title in titles:
        if title in a:
            return title

# Create new title column
candidates_df['title'] = candidates_df['candidate'].apply(lambda x: title_split(x))

# Fix existing candidate column name to remove title
candidates_df['candidate'] = candidates_df['candidate'].apply(lambda x: x.split(title_split(x))[1]).str.strip()

## Database Normalization

In [926]:
candidate_debates_df = pd.merge(candidates_df, debate_dialogue_df)


debates = candidate_debates_df.groupby(['date', 'type'], as_index=False).sum()

debates = debates.reset_index().rename(columns={'index':'debate_id'})

debates = debates[['debate_id', 'date', 'type']]

debates.to_csv('./Tables/debates.csv', index=False)

debates.head()

,debate_id,date,type
0,0,1960-09-26,P
1,1,1960-10-07,P
2,2,1960-10-13,P
3,3,1960-10-21,P
4,4,1976-09-23,P
5,5,1976-10-06,P
6,6,1976-10-22,P
7,7,1980-09-21,P
8,8,1980-10-28,P
9,9,1984-10-07,P


In [927]:
candidates_df.sort_values('year')

# candidates = candidates_df.groupby(['candidate', 'party'], as_index=False).sum()
candidates = candidates_df.drop_duplicates(subset='candidate')

candidates = candidates[['candidate', 'party']]

candidates = candidates.reset_index().rename(columns={'index':'candidate_id', 'candidate':'name'})

candidates.to_csv('./Tables/candidates.csv', index=False)

candidates.head()

,candidate_id,name,party
0,0,Richard Nixon,R
1,1,John F. Kennedy,D
2,2,Gerald Ford,R
3,3,Jimmy Carter,D
4,4,Bob Dole,R
5,5,Walter Mondale,D
6,7,Ronald Reagan,R
7,8,John B. Anderson,I
8,10,George H. W. Bush,R
9,12,Geraldine Ferraro,D


In [929]:
# Combine dataframes to get debate ids/types and dialogue stats
performances = pd.merge(debates, candidate_debates_df, on='date')

# Combine dataframes to get candidate ids and titles
performances = performances.merge(candidates, left_on='candidate', right_on='name')

# Filter dataframe to selected columns
performances = performances[['debate_id', 'candidate_id','type_x', 'title', 'speaking_turn_count', 'total_word_count', 'unique_word_count','won_election']].reset_index()

# Rename columns
performances = performances.rename(columns={'index':'performance_id','type_x':'type'})

# Create calculated column for avg number of words per turn
performances['words_per_turn'] = performances['total_word_count'] / performances['speaking_turn_count']

# Export performances table as csv
performances.to_csv('./Tables/performances.csv', index=False)
